In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
from crewai import Agent, Task, Crew
from dotenv import load_dotenv
from crewai import LLM
from langchain_community.llms import HuggingFaceHub
import os

load_dotenv()

llm = LLM(
    model="mistral/mistral-small-latest",  # ✅ Chat model
    api_key=os.getenv("MISTRAL_API_KEY")
)



## Creating Agents

- Define your Agents, and provide them a `role`, `goal` and `backstory`.
- It has been seen that LLMs perform better when they are role playing.

#### Planner Agent

In [3]:
planner = Agent(
    role = "Content Planner",
    goal = "Plan a content piece based on the provided {topic}.",
    backstory="You're working on planning a blog article "
              "about the topic: {topic}."
              "You collect information that helps the "
              "audience learn something "
              "and make informed decisions. "
              "Your work is the basis for "
              "the Content Writer to write an article on this topic.",   
    allow_delegation = False,
    verbose = True,
    llm = llm
)

#### Writer Agent


In [4]:
writer = Agent(
    role = "Content Writer",
    goal = "Write insightful and factually accurate "
         "opinion piece about the topic: {topic}",
    backstory="You're working on a writing "
              "a new opinion piece about the topic: {topic}. "
              "You base your writing on the work of "
              "the Content Planner, who provides an outline "
              "and relevant context about the topic. "
              "You follow the main objectives and "
              "direction of the outline, "
              "as provide by the Content Planner. "
              "You also provide objective and impartial insights "
              "and back them up with information "
              "provide by the Content Planner. "
              "You acknowledge in your opinion piece "
              "when your statements are opinions "
              "as opposed to objective statements.",
    allow_delegation=False,
    verbose=True,
    llm = llm
)

#### Editor Agent

In [5]:
editor = Agent(
    role="Editor",
    goal="Edit a given blog post to align with "
         "the writing style of the organization. ",
    backstory="You are an editor who receives a blog post "
              "from the Content Writer. "
              "Your goal is to review the blog post "
              "to ensure that it follows journalistic best practices,"
              "provides balanced viewpoints "
              "when providing opinions or assertions, "
              "and also avoids major controversial topics "
              "or opinions when possible.",
    allow_delegation=False,
    verbose=True,
    llm = llm
)

## Creating Tasks

- Define your Tasks, and provide them a `description`, `expected_output` and `agent`.

#### Plan Task

In [6]:
plan = Task(
    description=(
        "1. Prioritize the latest trends, key players, "
            "and noteworthy news on {topic}.\n"
        "2. Identify the target audience, considering "
            "their interests and pain points.\n"
        "3. Develop a detailed content outline including "
            "an introduction, key points, and a call to action.\n"
        "4. Include SEO keywords and relevant data or sources."
    ),
    expected_output="A comprehensive content plan document "
        "with an outline, audience analysis, "
        "SEO keywords, and resources.",
    agent=planner,
)

#### Write Task

In [7]:
write = Task(
    description=(
        "1. Use the content plan to craft a compelling "
            "blog post on {topic}.\n"
        "2. Incorporate SEO keywords naturally.\n"
		"3. Sections/Subtitles are properly named "
            "in an engaging manner.\n"
        "4. Ensure the post is structured with an "
            "engaging introduction, insightful body, "
            "and a summarizing conclusion.\n"
        "5. Proofread for grammatical errors and "
            "alignment with the brand's voice.\n"
    ),
    expected_output="A well-written blog post "
        "in markdown format, ready for publication, "
        "each section should have 2 or 3 paragraphs.",
    agent=writer,
)

#### Edit Task

In [8]:
edit = Task(
    description=("Proofread the given blog post for "
                 "grammatical errors and "
                 "alignment with the brand's voice."),
    expected_output="A well-written blog post in markdown format, "
                    "ready for publication, "
                    "each section should have 2 or 3 paragraphs.",
    agent=editor
)

## Creating the Crew

- Create your crew of Agents
- Pass the tasks to be performed by those agents.
    - **Note**: *For this simple example*, the tasks will be performed sequentially (i.e they are dependent on each other), so the _order_ of the task in the list _matters_.
- `verbose=2` allows you to see all the logs of the execution. 

In [9]:
crew = Crew(
    agents = [planner, writer, editor],
    tasks = [plan, write, edit],
    verbose = True,
)

## Runnning the Crew

**Note**: LLMs can provide different outputs for they same input, so what you get might be different than what you see in the video.

In [10]:
result = crew.kickoff(inputs = {"topic": "The Future of AI in Everyday Life"})

╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: c67293ab-9a76-4434-a2d2-e7d8597b482d                                                                       │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Content Planner                                                                                         │
│                                                                                                                 │
│  Task: 1. Prioritize the latest trends, key players, and noteworthy news on The Future of AI in Everyday Life.  │
│  2. Identify the target audience, considering their interests and pain points.                                  │
│  3. Develop a detailed content outline including an introduction, key points, and a call to action.             │
│  4. Include SEO keywords and relevant data or sources.                                                          │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Output()

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Content Planner                                                                                         │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  **Content Plan: The Future of AI in Everyday Life**                                                            │
│                                                                                                                 │
│  **1. Latest Trends, Key Players, and Noteworthy News**                                                         │
│                                                                                                                 │
│  **Latest Trends:**                                                                                             │
│  - AI in Smart Homes and IoT Devices                                                                            │
│  - AI-Powered Personal Assistants and Chatbots                                                                  │
│  - AI in Healthcare and Personalized Medicine                                                                   │
│  - AI in Education and E-Learning                                                                               │
│  - AI in Finance and Fintech                                                                                    │
│  - AI in Transportation and Autonomous Vehicles                                                                 │
│  - AI in Entertainment and Gaming                                                                               │
│  - AI in Cybersecurity and Privacy                                                                              │
│  - AI in Agriculture and Smart Farming                                                                          │
│  - AI in Retail and Customer Experience                                                                         │
│                                                                                                                 │
│  **Key Players:**                                                                                               │
│  - Tech Giants: Google, Apple, Microsoft, Amazon, IBM                                                           │
│  - AI Startups: OpenAI, DeepMind, SenseTime, Darktrace                                                          │
│  - Industry Leaders: Tesla (Autonomous Vehicles), IBM Watson (Healthcare), Ant Financial (Fintech)              │
│                                                                                                                 │
│  **Noteworthy News:**                                                                                           │
│  - AI advancements in drug discovery and personalized medicine                                                  │
│  - AI integration in smart cities and infrastructure                                                            │
│  - AI ethics and regulation debates                                                                             │
│  - AI's role in combating climate change and sustainability                                                     │
│  - AI's impact on jobs and the future of work                                                                   │
│                                                                                                                 │
│  **2. Target Audience Analysis**                                                                                │
│                                                        

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: 08b7ba61-86a1-470a-90c8-8d8205ce938c                                                                     │
│  Agent: Content Planner                                                                                         │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

...

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Editor                                                                                                  │
│                                                                                                                 │
│  Task: Proofread the given blog post for grammatical errors and alignment with the brand's voice.               │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Output()

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Editor                                                                                                  │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  ```markdown                                                                                                    │
│  # The Future of AI in Everyday Life                                                                            │
│                                                                                                                 │
│  ## Introduction                                                                                                │
│                                                                                                                 │
│  Artificial Intelligence (AI) has seamlessly integrated into our daily lives, from virtual assistants that      │
│  manage our schedules to recommendation algorithms that suggest our next favorite movie. As AI continues to     │
│  evolve, its impact on everyday life will only deepen. Understanding the future of AI helps us navigate its     │
│  benefits and challenges, ensuring we harness its potential responsibly. This article explores the latest       │
│  trends, key players, and the transformative role AI will play in various aspects of our lives.                 │
│                                                                                                                 │
│  AI is not just a technological advancement; it is a catalyst for change across multiple industries. From       │
│  healthcare to agriculture, AI's influence is profound and far-reaching. By examining the latest trends and     │
│  key players, we can better understand how AI is shaping our world and what the future holds.                   │
│                                                                                                                 │
│  ## AI in Smart Homes and IoT Devices                                                                           │
│                                                                                                                 │
│  AI-powered home assistants like Amazon Alexa and Google Home have become central to smart homes, managing      │
│  everything from lighting to security. These devices learn user preferences, automate routines, and even        │
│  predict needs, making homes more efficient and comfortable. AI's role in home security is also expanding,      │
│  with advanced systems capable of detecting unusual activity and alerting homeowners in real-time. Moreover,    │
│  AI contributes to energy efficiency by optimizing energy consumption based on usage patterns, reducing waste,  │
│  and lowering utility bills.                                                                                    │
│                                                                                                                 │
│  The Internet of Things (IoT) is another area where AI is making significant strides. AI algorithms analyze     │
│  data from IoT devices to improve decision-making and automation. For instance, smart thermostats like Nest     │
│  use AI to learn and adapt to users' heating and cooling preferences, enhancing comfort and energy savings. As  │
│  AI and IoT continue to converge, we can expect even more seamless and intuitive home environments.             │
│                                                                                                                 │
│  ## AI-Powered Personal Assistants and Chatbots        

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: e68da531-5724-40a4-85ea-f4eaba9b764c                                                                     │
│  Agent: Editor                                                                                                  │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name: crew                                                                                                     │
│  ID: c67293ab-9a76-4434-a2d2-e7d8597b482d                                                                       │
│  Tool Args:                                                                                                     │
│  Final Output: ```markdown                                                                                      │
│  # The Future of AI in Everyday Life                                                                            │
│                                                                                                                 │
│  ## Introduction                                                                                                │
│                                                                                                                 │
│  Artificial Intelligence (AI) has seamlessly integrated into our daily lives, from virtual assistants that      │
│  manage our schedules to recommendation algorithms that suggest our next favorite movie. As AI continues to     │
│  evolve, its impact on everyday life will only deepen. Understanding the future of AI helps us navigate its     │
│  benefits and challenges, ensuring we harness its potential responsibly. This article explores the latest       │
│  trends, key players, and the transformative role AI will play in various aspects of our lives.                 │
│                                                                                                                 │
│  AI is not just a technological advancement; it is a catalyst for change across multiple industries. From       │
│  healthcare to agriculture, AI's influence is profound and far-reaching. By examining the latest trends and     │
│  key players, we can better understand how AI is shaping our world and what the future holds.                   │
│                                                                                                                 │
│  ## AI in Smart Homes and IoT Devices                                                                           │
│                                                                                                                 │
│  AI-powered home assistants like Amazon Alexa and Google Home have become central to smart homes, managing      │
│  everything from lighting to security. These devices learn user preferences, automate routines, and even        │
│  predict needs, making homes more efficient and comfortable. AI's role in home security is also expanding,      │
│  with advanced systems capable of detecting unusual activity and alerting homeowners in real-time. Moreover,    │
│  AI contributes to energy efficiency by optimizing energy consumption based on usage patterns, reducing waste,  │
│  and lowering utility bills.                                                                                    │
│                                                                                                                 │
│  The Internet of Things (IoT) is another area where AI is making significant strides. AI algorithms analyze     │
│  data from IoT devices to improve decision-making and automation. For instance, smart thermostats like Nest     │
│  use AI to learn and adapt to users' heating and cooling preferences, enhancing comfort and energy savings. As  │
│  AI and IoT continue to converge, we can expect even more seamless and intuitive home environments.             │
│                                                       

## Key Points to Remember
- Agents perform better when role-playing.
- You should focus on goals and expectations.
- One agent can do multiple tasks.
- Tasks and agents should be granular.
- Tasks can be executed in different ways.

In [11]:
from IPython.display import Markdown
Markdown(result.raw)

```markdown
# The Future of AI in Everyday Life

## Introduction

Artificial Intelligence (AI) has seamlessly integrated into our daily lives, from virtual assistants that manage our schedules to recommendation algorithms that suggest our next favorite movie. As AI continues to evolve, its impact on everyday life will only deepen. Understanding the future of AI helps us navigate its benefits and challenges, ensuring we harness its potential responsibly. This article explores the latest trends, key players, and the transformative role AI will play in various aspects of our lives.

AI is not just a technological advancement; it is a catalyst for change across multiple industries. From healthcare to agriculture, AI's influence is profound and far-reaching. By examining the latest trends and key players, we can better understand how AI is shaping our world and what the future holds.

## AI in Smart Homes and IoT Devices

AI-powered home assistants like Amazon Alexa and Google Home have become central to smart homes, managing everything from lighting to security. These devices learn user preferences, automate routines, and even predict needs, making homes more efficient and comfortable. AI's role in home security is also expanding, with advanced systems capable of detecting unusual activity and alerting homeowners in real-time. Moreover, AI contributes to energy efficiency by optimizing energy consumption based on usage patterns, reducing waste, and lowering utility bills.

The Internet of Things (IoT) is another area where AI is making significant strides. AI algorithms analyze data from IoT devices to improve decision-making and automation. For instance, smart thermostats like Nest use AI to learn and adapt to users' heating and cooling preferences, enhancing comfort and energy savings. As AI and IoT continue to converge, we can expect even more seamless and intuitive home environments.

## AI-Powered Personal Assistants and Chatbots

Virtual assistants have evolved from simple voice-activated tools to sophisticated AI-powered companions. Siri, Google Assistant, and other AI assistants now understand context, manage complex tasks, and even anticipate user needs. These assistants are becoming more proactive, offering suggestions and taking actions before users explicitly ask, thanks to advanced machine learning algorithms.

In customer service, AI chatbots are revolutionizing support by providing instant, 24/7 assistance. These chatbots handle routine inquiries, freeing up human agents for more complex issues. AI's role in mental health is also gaining traction, with chatbots like Woebot offering therapeutic conversations and coping strategies. As AI technology advances, these assistants and chatbots will become even more integral to our daily lives, providing personalized support and enhancing productivity.

## AI in Healthcare and Personalized Medicine

AI is transforming healthcare by accelerating drug discovery and development. Machine learning algorithms analyze vast datasets to identify potential drug candidates, reducing the time and cost of bringing new treatments to market. AI-powered diagnostics and imaging tools enhance accuracy and speed, enabling earlier detection and treatment of diseases. For example, AI systems can analyze medical images to detect tumors or other abnormalities with high precision.

Personalized medicine is another area where AI is making a significant impact. By analyzing genetic, lifestyle, and environmental data, AI algorithms can tailor treatment plans to individual patients, improving outcomes and reducing side effects. AI's role in healthcare is poised to grow, with applications ranging from robotic surgery to remote patient monitoring, ultimately leading to more efficient and effective healthcare systems.

## AI in Education and E-Learning

AI is revolutionizing education by providing personalized learning experiences. AI tutors and adaptive learning platforms adjust to students' pacing and learning styles, offering customized content and feedback. This personalized approach helps students grasp concepts more effectively and at their own pace. AI also plays a crucial role in making education more accessible, with tools that assist students with disabilities, such as speech-to-text and text-to-speech technologies.

In administrative tasks, AI automates routine processes like grading and scheduling, freeing up educators' time to focus on teaching. AI-powered analytics can also identify at-risk students, allowing for early intervention and support. As AI continues to advance, it will further enhance the educational experience, making learning more engaging, inclusive, and effective.

## AI in Finance and Fintech

AI is reshaping the finance industry by improving fraud detection and risk assessment. Machine learning algorithms analyze transaction patterns to identify suspicious activities, helping financial institutions prevent fraud and protect customers. AI-powered investment and trading platforms use predictive analytics to offer personalized investment advice and automate trading strategies, making financial services more accessible and efficient.

In credit scoring and lending, AI algorithms evaluate a broader range of data points to assess creditworthiness, providing more accurate and inclusive lending decisions. AI's role in fintech is expanding, with applications in blockchain, digital payments, and regulatory compliance. As AI technology evolves, it will continue to drive innovation in the finance sector, enhancing security, efficiency, and customer experience.

## AI in Transportation and Autonomous Vehicles

AI is at the forefront of the autonomous vehicle revolution, with self-driving cars and public transportation systems becoming increasingly sophisticated. AI algorithms enable vehicles to navigate complex environments, make real-time decisions, and ensure passenger safety. AI's role in traffic management and urban planning is also growing, with smart city initiatives using AI to optimize traffic flow, reduce congestion, and improve public transportation.

AI contributes to sustainability by reducing emissions and promoting eco-friendly transportation options. For instance, AI-powered route optimization reduces fuel consumption and minimizes environmental impact. As autonomous vehicles and smart transportation systems become more prevalent, AI will play a pivotal role in creating safer, more efficient, and sustainable urban environments.

## AI in Entertainment and Gaming

AI is transforming the entertainment industry by enhancing content creation and curation. AI algorithms analyze viewer preferences to recommend personalized content, improving user experience on streaming platforms. In gaming, AI-powered non-player characters (NPCs) offer more realistic and engaging interactions, while AI-driven game design tools assist developers in creating immersive experiences.

Virtual and augmented reality (VR and AR) are also benefiting from AI advancements. AI enhances VR and AR experiences by enabling more realistic simulations, adaptive learning, and interactive storytelling. As AI continues to evolve, it will further revolutionize the entertainment and gaming industries, offering more personalized and immersive experiences.

## AI in Cybersecurity and Privacy

AI is crucial in cybersecurity, with advanced threat detection and prevention systems protecting against cyberattacks. Machine learning algorithms analyze network traffic and user behavior to identify and mitigate potential threats in real-time. AI's role in data protection and encryption is also expanding, with AI-powered tools ensuring secure data storage and transmission.

AI ethics and responsible innovation are critical considerations in cybersecurity. As AI systems become more sophisticated, ensuring they are used ethically and transparently is essential. AI's role in privacy protection will continue to grow, with applications in biometric authentication, secure communication, and data anonymization. By leveraging AI responsibly, we can create a safer and more secure digital world.

## AI in Agriculture and Smart Farming

AI is revolutionizing agriculture by enabling precision farming and crop monitoring. AI-powered drones and sensors collect data on soil health, crop conditions, and weather patterns, allowing farmers to make informed decisions and optimize yields. AI algorithms analyze this data to provide recommendations on planting, irrigation, and pest control, enhancing efficiency and sustainability.

In livestock management, AI tools monitor animal health and welfare, detecting early signs of illness and ensuring optimal care. AI's role in reducing food waste is also significant, with applications in supply chain optimization and quality control. As AI continues to advance, it will play a crucial role in creating a more sustainable and efficient agricultural system.

## AI in Retail and Customer Experience

AI is transforming the retail industry by enhancing personalized marketing and recommendations. AI algorithms analyze customer data to offer tailored product suggestions, improving the shopping experience and increasing sales. AI-powered inventory management and supply chain optimization ensure that retailers maintain optimal stock levels, reducing waste and improving efficiency.

AI's role in cashier-less stores and automated checkout systems is also expanding, providing faster and more convenient shopping experiences. AI-driven customer service chatbots offer instant support, addressing inquiries and resolving issues promptly. As AI technology evolves, it will continue to revolutionize the retail industry, creating more personalized and efficient shopping experiences.

## Conclusion

The future of AI in everyday life is bright and transformative. From smart homes and healthcare to education and retail, AI is poised to enhance efficiency, personalization, and sustainability. As AI continues to evolve, it is essential to stay informed about its advancements, ethical considerations, and potential impacts. By embracing AI responsibly, we can harness its potential to create a better, more connected world.

Understanding the future of AI is not just about technological advancements; it's about understanding how these advancements will shape our lives. By staying informed and engaged, we can ensure that AI is used ethically and responsibly, benefiting society as a whole.

## Call to Action

Share this article on social media to spread awareness about the future of AI in everyday life. Join the discussion by leaving a comment and sharing your thoughts on AI's potential and challenges. Explore further resources, such as AI courses, webinars, and industry reports, to deepen your understanding of this transformative technology.

By staying informed and engaged, we can all contribute to the responsible and ethical use of AI, ensuring that it benefits everyone.
```